In [3]:
import numpy as np
import jsonlines as jsl
import pandas as pd
import json
from flatten_json import flatten
import re
from sklearn.preprocessing import MinMaxScaler

# TODO


- define list of filters (range/value)
- preprocess all strings into ints, floats


## Filter List
price, location(map), highlights.living area, highlights.plot size, number of bedrooms, number of rooms, price per m^2, parking, in_the_neighborhood, construction.type_of_house, location?, year_of_construction, ...

In [4]:
path = './data/ads_trans.jsonlines'

In [5]:
reader = jsl.open(path)
first = reader.read()
first


{'crawl_time': 1685020679.2128732,
 'url': 'https://www.funda.nl/koop/amersfoort/huis-42126054-hogeweg-66/',
 'funda_identifier': '42126054',
 'images_paths': ['42126054/image3.jpeg',
  '42126054/image1.jpeg',
  '42126054/image4.jpeg',
  '42126054/image5.jpeg',
  '42126054/image2.jpeg'],
 'location_part1': 'Hogeweg 66',
 'location_part2': '3814 CE Amersfoort',
 'location_part3': 'Kruiskamp-Zuid',
 'price': '€ 450,000 k.k.',
 'highlights': {'living area': '110 m²',
  'plot size': '189 m²',
  'bedrooms': '2'},
 'features': {'transfer of ownership': {'asking price': '€ 450,000 kosten koper',
   'asking price per m²': "€ 4,091 The asking price per m² (€ 450,000) is calculated by dividing the asking price of the home by the living area (110 m²). Plot size and outbuildings aren't taken into account into this calculation. More information",
   'listed since': 'Today',
   'status': 'Available',
   'acceptance': 'Available in consultation'},
  'construction': {'kind of house': 'Single-family ho

In [6]:
data = []
file_path = path

# Read the .jsonlines file line by line
with open(file_path, 'r') as file:
    for line in file:        
        json_obj = json.loads(line)
        
        # Remove the 'images_paths' key and retrieve its value
        img_path = json_obj.pop('images_paths', '')
        neighb = json_obj.pop('in_the_neighborhood', '')
        
        json_obj = flatten(json_obj)
        json_obj['images_paths'] = img_path
        json_obj['in_the_neighborhood'] = neighb
        
        # Append the JSON object to the list
        data.append(json_obj)
        
        


In [7]:
df = pd.DataFrame.from_dict(data)
df.columns = df.columns.str.replace(' ', '_')

prefixes = ['features_', 'transfer_of_ownership_', 'construction_', 'surface_areas_and_volume_',
            'layout_', 'energy_', 'exterior_space_', 'storage_space_', 'parking_', 'geolocation_',
            'garage_', 'highlights_']

#remove prefixes created during flattening
for prefix in prefixes:
    df.columns = df.columns.str.replace(prefix, '')
    
#remove columns with duplicate names
df = df.loc[:, ~df.columns.duplicated()]

#remove irrelevant characteristics
df = df.filter(regex=r'^(?!vve_)')
df = df.drop(['asking_price','crawl_time', 'url', ], axis=1)


#remove properties without a set price
df['price'] = df['price'].str.replace(',', '').apply(lambda x: re.search(r'\d+', x).group(0) if re.search(r'\d+', x) else 'todrop')
df = df[df['price'] != 'todrop']
df['price'] = df['price'].astype(int)

#clean features with less than 1k non-empty values
df = df.dropna(axis=1, thresh=1000)
df = df.fillna('')

# df.head()


int_keys = [
    'living_area',
    'plot_size',
    'bedrooms',
    'year_of_construction',
    'volume_in_cubic_meters',
    'living_area',
    'external_storage_space',
    'number_of_rooms',
    'number_of_stories',
    'back_garden',
    'other_space_inside_the_building',
    'capacity',
]

sum_keys = [
        'number_of_bath_rooms'
]


float_keys=[
    'asking_price_per_m²',
    'lat',
    'lon'
]

for key in float_keys:
    df[key] = df[key].str.replace(',', '.').apply(lambda x: float(re.findall(r'\d+\.\d+|\d+', x)[0]) if re.findall(r'\d+\.\d+|\d+', x) else 0.0).astype(float)

for key in int_keys:
    df[key] = df[key].apply(lambda x: x.replace(',', '') if isinstance(x, str) else str(x))
    df[key] = df[key].apply(lambda x: int(re.search(r'\d+', x).group(0)) if re.search(r'\d+', x) else 0).astype(int)

for key in sum_keys:
    df[key] = df[key].apply(lambda x: x.replace(',', '') if isinstance(x, str) else str(x))
    df[key] = df[key].apply(lambda x: sum(map(int, re.findall(r'\d+', x))) if re.findall(r'\d+', x) else 0).astype(int)



In [8]:
# extract zipcode from location_part_2
first_two = df['location_part2'].str.slice(stop=2)
df['zip_code'] = first_two

df['building_age'] = 2023 - df['year_of_construction']

df['located_at'] = df['located_at'].apply(lambda x: re.findall(r'\d+', x)[0] if re.findall(r'\d+', x) else '0')
df['located_at'] = df['located_at'].replace(['Ground floor', ''], ['0', '0']).astype(int)


scaler = MinMaxScaler()
# Normalize the float values
df['lat_norm'] = scaler.fit_transform(df[['lat']])
df['lon_norm'] = scaler.fit_transform(df[['lon']])

df['garage_capacity'] = df['capacity']

df['has_garden'] = df['garden'].str.contains('garden', case=False)
df['has_patio'] = df['garden'].str.contains('patio', case=False)
df['has_terrace'] = df['garden'].str.contains('terrace', case=False)

df['has_roof'] = df['balcony/roof_garden'].str.contains('roof', case=False)
df['has_balcony'] = df['balcony/roof_garden'].str.contains('balcony', case=False)

# Mapping dictionary for substring to integer mapping
mapping = {'A+++++': 11, 'A++++': 10, 'A+++': 9, 'A++': 8, 'A+': 7, 'A': 6, 'B': 5, 'C': 4, 'D': 3, 'E': 2, 'F': 1}
# Apply the mapping dictionary to create a new integer column
df['label'] = df['label'].apply(lambda x: next((key for key in mapping if key in x), ''))

df['label_int'] = df['label'].apply(lambda x: next((mapping[key] for key in mapping if key in x), 0))


In [11]:
cats = ['zip_code','kind_of_house', 'building_type', 'label']
    
for key in cats:
    df[key] = df[key].astype('category')

In [8]:

# Modify the 'kind_of_house' column
df['kind_of_house'] = df['kind_of_house'].str.split(',', expand=True)[0]


array(['Resale property', nan, 'New property'], dtype=object)

In [62]:
df.to_csv('./data/dataset.csv', index=False)
